In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import warnings

import sys

sys.path.insert(0, "../code_workspace")

from data_prep import *
from post_analysis import *

warnings.filterwarnings("ignore")

/Users/suatakbulut/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<IPython.core.display.Javascript object>

In [3]:
use_saved_one = True
unwanted_column_endings = []
test_size = 0.4
models_path = "../trained_models/results_df_0.77_AUC_0.61_Rec.pkl"

<IPython.core.display.Javascript object>

In [4]:
X_train, X_test, y_train, y_test = prepare_train_test_split(
    use_saved_one=use_saved_one,
    unwanted_column_endings=unwanted_column_endings,
    test_size=test_size,
)


DATA PREP

Loading processed data from ../data/processed_dataset.pkl..
Excluding the following columns..
['patient_id', 'Outcome 30 days Hospitalization', 'Height(in)', 'Weight(lbs)']
Shapes: X_train: (108861, 136) - X_test: (72574, 136)


<IPython.core.display.Javascript object>

In [5]:
if not os.path.exists(models_path):
    raise ValueError(f"Trained model path {model_path} does not exist.")

results_df = pd.read_pickle(models_path)
results_df.set_index("classifier_class", drop=True)[
    ["AUC", "Recall", "Precision", "F1_Score", "Accuracy"]
]

,AUC,Recall,Precision,F1_Score,Accuracy
classifier_class,,,,,
RandomForestClassifier,0.769360,0.606889,0.674594,0.638952,0.706686
DecisionTreeClassifier,0.734679,0.566614,0.644530,0.603066,0.681015
GaussianNB,0.652868,0.560331,0.552130,0.556201,0.617590
QuadraticDiscriminantAnalysis,0.693367,0.521378,0.597497,0.556848,0.645107
GradientBoostingClassifier,0.750243,0.479879,0.695461,0.567899,0.687698
LogisticRegression,0.712588,0.449399,0.651015,0.531737,0.661504


<IPython.core.display.Javascript object>

In [6]:
best_model = (
    results_df[results_df.classifier_class == "RandomForestClassifier"]
    .trained_model.values[0]
    .best_estimator_
)

best_predictor = best_model.named_steps["predictor"]

<IPython.core.display.Javascript object>

In [8]:
_, _, feature_importance_labels = feature_importance_top10(best_model, X_test, y_test)
_, permutation_importance_labels = permutation_importance_top10(
    best_model, X_test, y_test
)

Elapsed time to compute feature importances: 0.00 mins


/Users/suatakbulut/miniconda3/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Elapsed time to compute permutation importances: 2.45 mins


<IPython.core.display.Javascript object>

In [9]:
imp_features = list(
    set(feature_importance_labels).union(set(permutation_importance_labels))
)

X_train = X_train[imp_features]
X_test = X_test[imp_features]

preprocessor = get_preprocessor()

X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

logit = sm.Logit(y_train, sm.add_constant(X_train)).fit()

Optimization terminated successfully.
         Current function value: 0.625158
         Iterations 5


<IPython.core.display.Javascript object>

In [10]:
print_logit_results(logit, X_train, X_test, y_train, y_test, imp_features)

                                  Logit Regression Results                                 
Dep. Variable:     Outcome 30 days Hospitalization   No. Observations:               108861
Model:                                       Logit   Df Residuals:                   108848
Method:                                        MLE   Df Model:                           12
Date:                             Mon, 14 Aug 2023   Pseudo R-squ.:                 0.08279
Time:                                     17:59:51   Log-Likelihood:                -68055.
converged:                                    True   LL-Null:                       -74199.
Covariance Type:                         nonrobust   LLR p-value:                     0.000
                                                   coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------------------------
const                                 

<IPython.core.display.Javascript object>

In [14]:
X_train.shape

(108861, 12)

<IPython.core.display.Javascript object>

### Re-fit the best model with the selected features only

In [11]:
best_predictor.fit(X_train, y_train)

auc_score, f_score, accuracy, precision, recall = calculate_metrics(
    best_predictor, X_test, y_test
)
auc_score_tr, f_score_tr, accuracy_tr, precision_tr, recall_tr = calculate_metrics(
    best_predictor, X_train, y_train
)





== Test Set vs Train Set ==
F1 Score  : 0.6083 | 0.8191
AUC Score : 0.7438 | 0.9331
Accuracy  : 0.6841 | 0.8539
Precision : 0.6475 | 0.8628
Recall    : 0.5736 | 0.7796



<IPython.core.display.Javascript object>

In [13]:
print("===========================")
print("== Test Set vs Train Set ==")
print(f"AUC Score : {auc_score:.4f} | {auc_score_tr:.4f}")
print(f"Recall    : {recall:.4f} | {recall_tr:.4f}")
print(f"F1 Score  : {f_score:.4f} | {f_score_tr:.4f}")
print(f"Accuracy  : {accuracy:.4f} | {accuracy_tr:.4f}")
print(f"Precision : {precision:.4f} | {precision_tr:.4f}")

== Test Set vs Train Set ==
AUC Score : 0.7438 | 0.9331
Recall    : 0.5736 | 0.7796
F1 Score  : 0.6083 | 0.8191
Accuracy  : 0.6841 | 0.8539
Precision : 0.6475 | 0.8628


<IPython.core.display.Javascript object>

In [15]:
cols = ["Forest on Test", "Forest on Train", "Logit on Test", "Logit on Train"]
ind = ["AUC", "Recall", "F1", "Accuracy", "Precision"]
data = [
    [0.7438, 0.9331, 0.6948, 0.6958],
    [0.5736, 0.7796, 0.4016, 0.4040],
    [0.6083, 0.8191, 0.4959, 0.4965],
    [0.6841, 0.8539, 0.6508, 0.6524],
    [0.6475, 0.8628, 0.6481, 0.6438],
]

comparison = pd.DataFrame(data, columns=cols, index=ind)
comparison

,Forest on Test,Forest on Train,Logit on Test,Logit on Train
AUC,0.7438,0.9331,0.6948,0.6958
Recall,0.5736,0.7796,0.4016,0.4040
F1,0.6083,0.8191,0.4959,0.4965
Accuracy,0.6841,0.8539,0.6508,0.6524
Precision,0.6475,0.8628,0.6481,0.6438


<IPython.core.display.Javascript object>

In [28]:
dd = (
    results_df.set_index("classifier_class", drop=True)[
        ["AUC", "Recall", "Precision", "F1_Score", "Accuracy"]
    ]
    .loc[["RandomForestClassifier", "LogisticRegression"]]
    .transpose()
)
dd

classifier_class,RandomForestClassifier,LogisticRegression
AUC,0.769360,0.712588
Recall,0.606889,0.449399
Precision,0.674594,0.651015
F1_Score,0.638952,0.531737
Accuracy,0.706686,0.661504


<IPython.core.display.Javascript object>